In [6]:
import sys
from pathlib import Path
import json
from collections import OrderedDict

import folium as fl
from json2html import json2html as jh
import shapely.geometry as sg

sys.path.append('../')

import gtfstk as gt

DATA_DIR = Path('../data')

In [7]:
path = DATA_DIR/'cairns_gtfs.zip'
feed = gt.read_gtfs(path, dist_units='km')
feed.describe()

,indicator,value
0,agencies,[Department of Transport and Main Roads - Tran...
1,timezone,Australia/Brisbane
2,start_date,20140526
3,end_date,20141228
4,num_routes,22
5,num_trips,1339
6,num_stops,416
7,num_shapes,54
8,sample_date,20140529
9,num_routes_active_on_sample_date,20


In [14]:
COLORS_10 = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd',
  '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']
COLORS_SET2 = ['#66c2a5','#fc8d62','#8da0cb','#e78ac3','#a6d854','#ffd92f','#e5c494','#b3b3b3']

# Move this to GTFSTK
def route_to_geojson(feed, route_id, date=None, *, include_stops=False):
    """
    """
    # Get set of unique trip shapes for route
    shapes = (
        feed.get_trips(date=date)
        .loc[lambda x: x['route_id'] == route_id, 'shape_id']
        .unique()
    )
    geom_by_shape = feed.build_geometry_by_shape(shape_ids=shapes)
    
    # Get route properties
    route = (
        feed.get_routes(date=date)
        .loc[lambda x: x['route_id'] == route_id]
        .fillna('n/a')
        .to_dict(orient='records', into=OrderedDict)
    )[0]
       
    # Build route shape features
    features = [{
        'type': 'Feature',
        'properties': route,
        'geometry': sg.mapping(sg.LineString(geom)),
    } for geom in geom_by_shape.values() ]
    
    # Build stop features if desired
    if include_stops:
        stops = (
            feed.get_stops(route_id=route_id)
            .fillna('n/a')
            .to_dict(orient='records', into=OrderedDict)
        )
        features.extend([{
            'type': 'Feature',
            'geometry': {
                'type': 'Point',
                'coordinates': [stop['stop_lon'], stop['stop_lat']],
              },
            'properties': stop,
        } for stop in stops])
    
    return {'type': 'FeatureCollection', 'features': features}

def make_html(prop):
    return jh.convert(prop, table_attributes="class=\"table table-condensed table-hover\"")
    
def map_routes(feed, route_short_names=None, route_ids=None, *, include_stops=True):
    
    if route_ids is not None:
        cond = lambda x: x['route_id'].isin(route_ids)
    elif route_short_names is not None:
        cond = lambda x: x['route_short_name'].isin(route_short_names)
    else:
        raise ValueError("One of route_ids or route_short_names must be given")
        
    # Get routes DataFrame slice and convert to dictionary
    routes = feed.routes.loc[cond].fillna('n/a').to_dict(orient='records')
    
    # Initialize map
    my_map = fl.Map(tiles='cartodbpositron')
    
    # Choose colors
    n = len(routes)
    palette = COLORS_SET2
    colors = [palette[i % len(palette)] for i in range(n)]
    
    # For each route, add its path and stops to the map as a feature group
    for i, route in enumerate(routes):
        collection = route_to_geojson(feed, route['route_id'], include_stops=include_stops)
        group = fl.FeatureGroup(name='Route ' + route['route_short_name'])
        color = colors[i]
        for f in collection['features']:
            prop = f['properties']
            
            # Add stop
            if f['geometry']['type'] == 'Point':
                lon, lat = f['geometry']['coordinates']
                fl.CircleMarker(
                    location=[lat, lon],
                    radius=8,
                    fill=True,
                    color=color,
                    weight=1,
                    popup=fl.Popup(make_html(prop))
                ).add_to(group)
            
            # Add path
            else:
                prop['color'] = color
                path = fl.GeoJson(f, 
                    name=route,
                    style_function=lambda x: {'color': x['properties']['color']},
                )
                path.add_child(fl.Popup(make_html(prop)))
                path.add_to(group)

        group.add_to(my_map)
        
    fl.LayerControl().add_to(my_map)
    my_map.location = [lat, lon]
    my_map.zoom_start = 11
    
    # Maybe add a legend with the route colors and route short names.
    # Legends are an open issue: https://github.com/python-visualization/folium/issues/528.
    # Here's a hack: https://medium.com/@bobhaffner/creating-a-legend-for-a-folium-map-c1e0ffc34373.
    
    return my_map



In [15]:
rsns = feed.routes.route_short_name.tolist()[:2]
%time map_routes(feed, rsns)


CPU times: user 3.53 s, sys: 19.9 ms, total: 3.55 s
Wall time: 3.55 s


In [5]:
feed.routes

,route_id,route_short_name,route_long_name,route_desc,route_type,route_url,route_color,route_text_color
0,110-423,110,City - Palm Cove,NaN,3,NaN,7BC142,000000
1,110N-423,110N,City - Palm Cove,NaN,3,NaN,7BC142,000000
2,111-423,111,City - Kewarra Beach,NaN,3,NaN,7BC142,000000
3,112-423,112,Yorkeys Knob - Smithfield via JCU,NaN,3,NaN,7BC142,000000
4,113-423,113,Sunbus Depot - Cairns City Mall,NaN,3,NaN,7BC142,000000
5,120-423,120,City Pier - Machans Beach - Smithfield,NaN,3,NaN,7BC142,000000
6,120N-423,120N,City - Smithfield via Machans Beach and Holloways,NaN,3,NaN,7BC142,000000
7,121-423,121,City - Redlynch,NaN,3,NaN,7BC142,000000
8,122-423,122,JCU - Redlynch,NaN,3,NaN,7BC142,000000
9,123-423,123,City - JCU via Raintrees,NaN,3,NaN,7BC142,000000
